<a href="https://colab.research.google.com/github/shicong0720/Finding-Healthy-Community-Design-Criteria-with-AI/blob/main/02_21_ANN__combine_PCA_adjusted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import os
import numpy as np
import random
from google.colab import drive

In [2]:
drive.mount('/content/gdrive',force_remount=True)
os.chdir("/content/gdrive/My Drive/big data and city/")

Mounted at /content/gdrive


In [60]:

import pandas as pd

data = pd.read_csv('combine_train_21.csv',header=None)
data = data.values.tolist()
test = pd.read_csv('combine_test_21.csv',header=None)
test = test.values.tolist()

maxi = [-99999999999 for i in range(len(data[0]))]
mini = [99999999999 for i in range(len(data[0]))]

for i in range(len(data)):
    for j in range(len(data[i])):
        if data[i][j]>maxi[j]:
            maxi[j] = data[i][j]
        if data[i][j]<mini[j]:
            mini[j] = data[i][j]

#maxi = [100000000000,200000000000,20]
#mini = [0,0,0]

for i in range(len(data)):
    for j in range(len(data[i])):
        data[i][j] = (data[i][j]-mini[j])/(maxi[j]-mini[j])


for i in range(len(test)):
    for j in range(len(test[i])):
        test[i][j] = (test[i][j]-mini[j])/(maxi[j]-mini[j])

In [44]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

def initialize():
    #placeholder
    x = tf.placeholder(tf.float32, [None, D_in])
    y = tf.placeholder(tf.float32, [None, D_out])
    w = {
        'h1': tf.Variable(tf.random_normal([D_in, D_h1])),
        'h2': tf.Variable(tf.random_normal([D_h1, D_h2])),
        'h3': tf.Variable(tf.random_normal([D_h2, D_h3])),
        'h4': tf.Variable(tf.random_normal([D_h3, D_h4])),
        'h5': tf.Variable(tf.random_normal([D_h4, D_h5])),
        'h6': tf.Variable(tf.random_normal([D_h5, D_h6])),
        'h7': tf.Variable(tf.random_normal([D_h6, D_h7])),
        'h8': tf.Variable(tf.random_normal([D_h7, D_h8])),
        'h9': tf.Variable(tf.random_normal([D_h8, D_h9])),
        'h10': tf.Variable(tf.random_normal([D_h9, D_h10])),
        'out': tf.Variable(tf.random_normal([D_h10, D_out]))
    }
    b = {
        'h1': tf.Variable(tf.random_normal([D_h1])),
        'h2': tf.Variable(tf.random_normal([D_h2])),
        'h3': tf.Variable(tf.random_normal([D_h3])),
        'h4': tf.Variable(tf.random_normal([D_h4])),
        'h5': tf.Variable(tf.random_normal([D_h5])),
        'h6': tf.Variable(tf.random_normal([D_h6])),
        'h7': tf.Variable(tf.random_normal([D_h7])),
        'h8': tf.Variable(tf.random_normal([D_h8])),
        'h9': tf.Variable(tf.random_normal([D_h9])),
        'h10': tf.Variable(tf.random_normal([D_h10])),
        'out': tf.Variable(tf.random_normal([D_out]))
    }

    #activation functions #tf.layers.batch_normalization()
    def multilayer_perceptron(x):
        h1_layer = tf.sigmoid(tf.add(tf.matmul(x, w['h1']), b['h1']))
        h2_layer = tf.sigmoid(tf.add(tf.matmul(h1_layer, w['h2']), b['h2']))
        h3_layer = tf.sigmoid(tf.add(tf.matmul(h2_layer, w['h3']), b['h3']))
        h4_layer = tf.sigmoid(tf.add(tf.matmul(h3_layer, w['h4']), b['h4']))
        h5_layer = tf.sigmoid(tf.add(tf.matmul(h4_layer, w['h5']), b['h5']))
        h6_layer = tf.sigmoid(tf.add(tf.matmul(h5_layer, w['h6']), b['h6']))
        h7_layer = tf.sigmoid(tf.add(tf.matmul(h6_layer, w['h7']), b['h7']))
        h8_layer = tf.sigmoid(tf.add(tf.matmul(h7_layer, w['h8']), b['h8']))
        h9_layer = tf.sigmoid(tf.add(tf.matmul(h8_layer, w['h9']), b['h9']))
        h10_layer = tf.sigmoid(tf.add(tf.matmul(h9_layer, w['h10']), b['h10']))
        out_layer = tf.sigmoid(tf.add(tf.matmul(h10_layer, w['out']), b['out']))
        return out_layer
    pred = multilayer_perceptron(x)

    #loss function
    #cost = tf.reduce_sum(tf.where(tf.greater(pred, 0.932), 10*(y-pred)*(y-pred),
    #                     tf.where(tf.greater(0.042, pred), 10*(y-pred)*(y-pred),
    #                     (y-pred)*(y-pred))))
    cost = tf.reduce_sum((y-pred)*(y-pred))

    #optimizer and others
    optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
    init = tf.global_variables_initializer()
    variables_dict = {
        'b1': b['h1'],
        'b2': b['h2'],
        'b3': b['h3'],
        'b4': b['h4'],
        'b5': b['h5'],
        'b6': b['h6'],
        'b7': b['h7'],
        'b8': b['h8'],
        'b9': b['h9'],
        'b10': b['h10'],
        'bout': b['out'],
        'w1': w['h1'],
        'w2': w['h2'],
        'w3': w['h3'],
        'w4': w['h4'],
        'w5': w['h5'],
        'w6': w['h6'],
        'w7': w['h7'],
        'w8': w['h8'],
        'w9': w['h9'],
        'w10': w['h10'],
        'wout': w['out']
    }
    saver = tf.train.Saver(variables_dict)
    saver = tf.train.Saver(max_to_keep=1000, keep_checkpoint_every_n_hours=1)
    return x, y, pred, cost, optimizer, init, saver


def evaluate(filename, dimensionx):
    xdata = []
    with open(filename) as f:
        lines = f.readlines()
    temp = []
    for line in lines:
        temp.append(float(line))
        if len(temp)==dimensionx:
            xdata.append(temp)
            temp = []
    
    with tf.Session() as sess:
        saver.restore(sess, "./model.ckpt")
        yhat = pred.eval(feed_dict = {x: xdata})
        
        
def evaluatewhentrain(xdata, modelfile):
    with tf.Session() as sess:
        saver.restore(sess, modelfile)
        yhat = pred.eval(feed_dict = {x: xdata})
    return yhat


def train(X_test, Y_test):
    with tf.Session() as sess:
        sess.run(init)
        merged_all = tf.summary.merge_all()
        writer = tf.summary.FileWriter('logs/', sess.graph)
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(num/batch_size)
            if num%batch_size != 0:
                total_batch = total_batch + 1
            for i in range(total_batch):
                batch_xs = xdata[i*batch_size:(i+1)*batch_size]
                batch_ys = ydata[i*batch_size:(i+1)*batch_size]
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs, y: batch_ys})
                avg_cost += c / total_batch
            if (epoch+1) % display_step == 0 or epoch == 0:
                print("Epoch:", '%04d' % (epoch+1), " cost=", "{:.9f}".format(avg_cost))
                #merged = sess.run(merged_all, feed_dict={x: batch_xs, y: batch_ys})
                #writer.add_summary(merged, epoch)
            if (epoch+1) % save_step == 0:
                saver.save(sess, "./model"+str(epoch)+".ckpt")
                print("save model done.")
                yhat = np.array(evaluatewhentrain(X_test,"./model"+str(epoch)+".ckpt"))
                accuracy = 100 - np.around(np.median(np.abs(yhat-Y_test)*100,axis=0),decimals=1)
                
                print('10-layer ANN', accuracy, '% Step '+str(epoch))

              

In [93]:
#training settings
training_epochs = 200000
save_step = 1000

batch_size = 100
display_step = 1000

#network structure
D_in, D_out = 32, 18
D_h1 = 32
D_h2 = 32
D_h3 = 32
D_h4 = 32
D_h5 = 32
D_h6 = 32
D_h7 = 32
D_h8 = 32
D_h9 = 32
D_h10 = 32

#initialize
x, y, pred, cost, optimizer, init, saver = initialize()

#load data
xdata = np.array(data)[:,0:32]
ydata = np.array(data)[:,32:]
X_test = np.array(test)[:,0:32]
Y_test = np.array(test)[:,32:]

num = len(xdata)
print(xdata)
print(ydata)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  9.42827419e-04 2.86608123e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  2.00173451e-03 7.67276577e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.53475696e-01
  7.87908754e-02 1.98964572e-03]
 ...
 [5.97014925e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  2.41530466e-03 1.35842231e-03]
 [1.13432836e-02 4.00000000e-03 0.00000000e+00 ... 2.67566760e-01
  2.67860439e-02 7.90637644e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  6.76863764e-04 2.58131879e-04]]
[[0.03069054 0.39055794 0.43283582 ... 0.26732673 0.19512195 0.03492063]
 [0.40409207 0.33261803 0.5        ... 0.62376238 0.29268293 0.35555556]
 [0.12787724 0.35622318 0.52238806 ... 0.56930693 0.25609756 0.14603175]
 ...
 [0.0511509  0.33690987 0.5        ... 0.2970297  0.17073171 0.04761905]
 [0.04347826 0.36695279 0.27238806 ... 0.23762376 0.2195122  0.04126984]
 [0.59079284 0.27682403 0.4291

In [62]:
train(X_test, Y_test)

Epoch: 0001  cost= 359.280957540
Epoch: 1000  cost= 24.024445375
save model done.
INFO:tensorflow:Restoring parameters from ./model999.ckpt
5-layer ANN [61.8 86.1 90.1 88.3 94.  82.7 85.5 93.2 83.7 85.7 75.1 86.4 80.1 79.5
 79.7 82.3 83.9 71.6] % Step 999
Epoch: 2000  cost= 21.027596951
save model done.
INFO:tensorflow:Restoring parameters from ./model1999.ckpt
5-layer ANN [68.2 87.4 90.3 89.  94.7 85.1 86.2 93.9 85.8 86.8 77.8 86.9 81.9 80.9
 81.4 83.  85.5 75.3] % Step 1999
Epoch: 3000  cost= 18.709385316
save model done.
INFO:tensorflow:Restoring parameters from ./model2999.ckpt
5-layer ANN [71.2 88.7 90.6 89.6 95.3 86.4 87.5 94.2 87.1 88.1 79.8 87.7 84.8 83.5
 84.5 84.8 86.8 78.7] % Step 2999
Epoch: 4000  cost= 16.927430948
save model done.
INFO:tensorflow:Restoring parameters from ./model3999.ckpt
5-layer ANN [73.7 89.2 91.1 90.8 95.5 87.5 87.4 94.7 88.4 89.3 82.  89.1 84.9 84.3
 86.1 85.8 88.3 80.3] % Step 3999
Epoch: 5000  cost= 15.875821908
save model done.
INFO:tensorflow:Rest

In [97]:
train(X_test, Y_test)

Epoch: 0001  cost= 269.170731862
Epoch: 1000  cost= 22.808909416
save model done.
INFO:tensorflow:Restoring parameters from ./model999.ckpt
5-layer ANN [44.1 82.8 89.7 86.2 91.6 74.7 74.9 89.4 77.  79.6 55.8 79.7 64.  63.
 60.5 71.5 76.7 51.8] % Step 999
Epoch: 2000  cost= 19.889841557
save model done.
INFO:tensorflow:Restoring parameters from ./model1999.ckpt
5-layer ANN [45.9 80.3 89.3 85.  90.6 70.8 73.  88.3 76.2 77.7 52.5 80.1 61.4 59.6
 59.5 69.8 76.9 44.8] % Step 1999
Epoch: 3000  cost= 17.668447415
save model done.
INFO:tensorflow:Restoring parameters from ./model2999.ckpt
5-layer ANN [50.7 82.1 88.8 84.5 91.4 71.5 74.4 88.7 76.9 78.  49.9 79.3 61.7 61.3
 59.  70.1 75.  45.4] % Step 2999
Epoch: 4000  cost= 16.242840370
save model done.
INFO:tensorflow:Restoring parameters from ./model3999.ckpt
5-layer ANN [48.6 81.3 88.8 83.1 91.3 72.1 70.5 88.4 75.9 75.9 51.1 78.5 60.6 61.5
 56.5 70.4 73.4 45.9] % Step 3999
Epoch: 5000  cost= 14.905968428
save model done.
INFO:tensorflow:Resto

In [99]:
X_pre = X_test

with tf.Session() as sess:
    saver.restore(sess, "./model99999.ckpt")
    yhat = pred.eval(feed_dict = {x: X_pre})

print(yhat)

INFO:tensorflow:Restoring parameters from ./model99999.ckpt
[[0.10939056 0.54100937 0.3895779  ... 0.3454631  0.39846292 0.18927112]
 [0.13264737 0.29210526 0.6346607  ... 0.34112376 0.17961672 0.14987347]
 [0.11052015 0.5770533  0.45068058 ... 0.38272235 0.41620266 0.22650334]
 ...
 [0.07351556 0.40295908 0.60283756 ... 0.27522022 0.2246432  0.0897975 ]
 [0.30248126 0.23400852 0.6356312  ... 0.89794993 0.18822888 0.2749098 ]
 [0.03447852 0.50463074 0.44666705 ... 0.18960026 0.26181817 0.05624428]]


In [101]:
reader = tf.train.NewCheckpointReader('model99999.ckpt')
all_variables = reader.get_variable_to_shape_map()
print(all_variables)
w = reader.get_tensor("Variable")
print(type(w))
print(w.shape)
print(w)

{'beta2_power_7': [], 'beta2_power_5': [], 'beta2_power_11': [], 'beta2_power_10': [], 'beta1_power_9': [], 'beta1_power_7': [], 'beta1_power_6': [], 'beta1_power_4': [], 'beta1_power_2': [], 'beta1_power_16': [], 'beta1_power_10': [], 'beta1_power': [], 'Variable_99/Adam_1': [32], 'Variable_96/Adam_1': [32, 32], 'Variable_96/Adam': [32, 32], 'Variable_96': [32, 32], 'Variable_95/Adam_1': [32, 32], 'Variable_95': [32, 32], 'Variable_94/Adam_1': [32, 32], 'Variable_93': [32, 32], 'Variable_92/Adam_1': [32, 32], 'Variable_92/Adam': [32, 32], 'Variable_92': [32, 32], 'Variable_91': [32, 32], 'Variable_9/Adam_1': [32, 32], 'Variable_9': [32, 32], 'Variable_89/Adam': [32, 32], 'Variable_89': [32, 32], 'Variable_88/Adam': [32, 32], 'Variable_87': [16], 'Variable_86/Adam_1': [32], 'Variable_85/Adam_1': [32], 'Variable_85': [32], 'Variable_86': [32], 'Variable_83/Adam': [32], 'Variable_80/Adam': [32], 'Variable_80': [32], 'Variable_8/Adam_1': [32, 32], 'Variable_79': [32], 'Variable_77': [32],

In [103]:
import warnings
warnings.filterwarnings("ignore")
import datetime
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import sys
import random
%matplotlib inline


X_train = xdata
y_train = ydata


y_test = Y_test


# import xgboost as xgb
from sklearn.model_selection import train_test_split


# Different types of regressors
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

def my_custom_loss_func(ground_truth, predictions):
    return np.average(np.around(np.median(100*np.abs(predictions-ground_truth),axis=0),decimals=1))

#outtemp = []
#exdata, exdatastr = exeva(0,[],'',10)

n_estimators = 300
regr = DecisionTreeRegressor()
regr.fit(X_train,y_train)
#outtemp.append(findDT(regr,exdata,exdatastr))
#evadata('data/nne.txt', D_in, regr)
preds = regr.predict(X_test)
print ('Median Accuracy for DecisionTreeRegressor',100-my_custom_loss_func(y_test,preds),'%')

regr = BaggingRegressor(DecisionTreeRegressor(), n_estimators=n_estimators,n_jobs=-1)
regr.fit(X_train,y_train)
#outtemp.append(findDT(regr,exdata,exdatastr))
#evadata('data/nne.txt', D_in, regr)
preds = regr.predict(X_test)
print ('Median Accuracy for BaggingRegressor', 100-my_custom_loss_func(y_test,preds),'%')

regr = RandomForestRegressor(n_estimators=n_estimators, max_depth=None,min_samples_split=2, random_state=0,n_jobs=-1)
regr.fit(X_train,y_train)
#outtemp.append(findDT(regr,exdata,exdatastr))
#evadata('data/nne.txt', D_in, regr)
preds = regr.predict(X_test)
print ('Median Accuracy for RandomForestRegressor', 100-my_custom_loss_func(y_test,preds),'%')

regr = ExtraTreesRegressor(n_estimators=n_estimators, max_depth=None,min_samples_split=2, random_state=0,n_jobs=-1)
regr.fit(X_train,y_train)
#outtemp.append(findDT(regr,exdata,exdatastr))
#evadata('data/nne.txt', D_in, regr)
preds = regr.predict(X_test)
print ('Median Accuracy for ExtraTreesRegressor', 100-my_custom_loss_func(y_test,preds),'%')

reg = LinearRegression()
reg.fit(X_train,y_train)
preds = reg.predict(X_test)
print ('Median Accuracy for LinearRegressor', 100-my_custom_loss_func(y_test,preds),'%')

print ('Median Accuracy for ANN', 100-my_custom_loss_func(y_test,yhat),'%')

Median Accuracy for DecisionTreeRegressor 90.56666666666666 %
Median Accuracy for BaggingRegressor 91.82777777777778 %
Median Accuracy for RandomForestRegressor 91.74444444444444 %
Median Accuracy for ExtraTreesRegressor 91.32222222222222 %
Median Accuracy for LinearRegressor 90.45555555555555 %
Median Accuracy for ANN 87.5111111111111 %
